In [1]:
import pandas as pd
import re
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.optimizers import Adam

In [2]:
df = pd.read_csv("cb_tweets.csv")

In [3]:
df = df.rename(columns={"tweet_text": "text", "cyberbullying_type": "label"})
df['label'] = df['label'].apply(lambda x: 0 if x == 'not_cyberbullying' else 1)

def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'http\S+', '', text)        # remove URLs
    text = re.sub(r'@\w+', '', text)           # remove mentions
    text = re.sub(r'#\w+', '', text)           # remove hashtags
    text = re.sub(r'[^a-z\s]', '', text)       # remove punctuation/numbers
    text = re.sub(r'\s+', ' ', text).strip()   # remove extra spaces
    return text
df['clean_text'] = df['text'].apply(clean_text)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    df['clean_text'], df['label'], test_size=0.2, random_state=42, stratify=df['label']
)

MAX_VOCAB = 10000
MAX_LEN = 50

tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

In [5]:
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post', truncating='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post', truncating='post')

In [6]:
model = Sequential([
    Embedding(input_dim=MAX_VOCAB, output_dim=128, input_length=MAX_LEN),
    Conv1D(filters=128, kernel_size=5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

C:\Users\sauravsuman01\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d (Conv1D)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d                 │ ?                           │               0 │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [7]:
history = model.fit(
    X_train_pad,
    y_train,
    validation_split=0.2,
    epochs=5,
    batch_size=64,
    verbose=1
)

Epoch 1/5
477/477 ━━━━━━━━━━━━━━━━━━━━ 11s 20ms/step - accuracy: 0.8471 - loss: 0.3096 - val_accuracy: 0.8589 - val_loss: 0.2668
Epoch 2/5
477/477 ━━━━━━━━━━━━━━━━━━━━ 11s 22ms/step - accuracy: 0.8879 - loss: 0.2292 - val_accuracy: 0.8621 - val_loss: 0.2770
Epoch 3/5
477/477 ━━━━━━━━━━━━━━━━━━━━ 11s 23ms/step - accuracy: 0.9185 - loss: 0.1756 - val_accuracy: 0.8564 - val_loss: 0.3357
Epoch 4/5
477/477 ━━━━━━━━━━━━━━━━━━━━ 18s 18ms/step - accuracy: 0.9431 - loss: 0.1266 - val_accuracy: 0.8489 - val_loss: 0.4650
Epoch 5/5
477/477 ━━━━━━━━━━━━━━━━━━━━ 13s 23ms/step - accuracy: 0.9575 - loss: 0.0938 - val_accuracy: 0.8416 - val_loss: 0.4511


In [8]:
y_pred_prob = model.predict(X_test_pad)
y_pred = (y_pred_prob > 0.5).astype("int32")

print("\n📊 Classification Report:")
print(classification_report(y_test, y_pred, target_names=["non-bullying", "bullying"]))

acc = accuracy_score(y_test, y_pred)
print(f"✅ Test Accuracy: {acc:.4f}")

299/299 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step

📊 Classification Report:
              precision    recall  f1-score   support

non-bullying       0.51      0.50      0.50      1589
    bullying       0.90      0.91      0.90      7950

    accuracy                           0.84      9539
   macro avg       0.71      0.70      0.70      9539
weighted avg       0.83      0.84      0.84      9539

✅ Test Accuracy: 0.8369


The CNN model achieved an overall accuracy of 84%, showing strong performance in detecting bullying tweets with a precision of 0.90, recall of 0.91, and F1-score of 0.90. This indicates that the model accurately identifies harmful content while minimizing false negatives. For the non-bullying class, the precision and recall are lower (around 0.50), showing moderate performance due to dataset imbalance. The macro average F1-score of 0.70 suggests fair balance between both classes, while the weighted average of 0.84 reflects dominance of the majority class. Overall, the CNN effectively captures text patterns and context for cyberbullying detection.